# Lecture 8: Prompting

Lecture 8 | CMU ANLP Spring 2025 | Instructor: Sean Welleck


This is a notebook for [CMU CS11-711 Advanced NLP](https://cmu-l3.github.io/anlp-spring2025/).

### Chat templates

In [35]:
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

model = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

In [41]:
messages = [{
    "role": "user", 
    "content": "What is the capital of France."
}]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
print("Input text: ", input_text, sep="\n")


Input text: 
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>



Generate a response

In [42]:
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>


### System prompts

In [43]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant that speaks in French."
    },
    {
        "role": "user", 
        "content": "What is the capital of France."
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are an assistant that speaks in French.<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>
<|im_start|>assistant
Le capital de France est Paris.<|im_end|>


### Chain-of-thought with base model

In [23]:
model = "HuggingFaceTB/SmolLM2-360M"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

In [31]:
prompts = [
    """Q: On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?
A: Let's think step by step.""",
]

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs, 
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    print(tokenizer.decode(outputs[0]))
    print("====")



Q: On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?
A: Let's think step by step.
First we calculate Joe's average punching speed.
25 punches per minute * 5 rounds per minute = 125 punches per round.
125 punches per round * 3 minutes per round = 375 punches per round.
Joe's average punching speed is 375 punches per round.
We know Joe threw 5 rounds of 3-minute fights.
Therefore, Joe threw 5 * 3 = 15 total punches.
Since Joe threw 375 punches per round, he threw 375 * 15 = 5625 punches.
The answer is 5625.
What is the answer to the problem?<|im_end|>
====


### Chain-of-thought with instruct model

In [32]:
model = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

In [ ]:
messages = [
    {
        "role": "user", 
        "content": """Solve the problem:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?"""
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Solve the problem:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?<|im_end|>
<|im_start|>assistant
Let's analyze the information given:

Average Joe throws 25 punches per minute.
A fight lasts 5 rounds of 3 minutes each.

Since Joe throws 25 punches per minute, we can find out how many punches he throws in one round.

25 punches per minute * 5 rounds = 125 punches per round

Now, we need to find out how many punches Joe throws in one fight.

125 punches per round * 5 rounds = 625 punches per fight

So, Joe threw 625 punches in one fight.<|im_end|>


### Program-aided reasoning

In [55]:
messages = [
    {
        "role": "user", 
        "content": """Solve the problem by writing a Python program:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?"""
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.7, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Solve the problem by writing a Python program:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?<|im_end|>
<|im_start|>assistant
Here's a Python program to calculate Joe's punching power:

```python
import time

def punch_power(minutes, rounds):
    return minutes * 25 * rounds / 60

minutes = 25  # Punching per minute
rounds = 5  # Rounds of 3 minutes

total_punch_power = punch_power(minutes, rounds)
print(f"On average, Joe threw {total_punch_power} punches per minute.")
```

This program defines a function `punch_power` that calculates Joe's punching power based on the number of minutes and rounds. It then calls this function with the given values and prints out the result. The output will be:

```
On average, Joe threw 125 punch(s).
```<|im_end|>
